# Slackbot-related work

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from trulens_evalchain.tru_db import Record, LocalTinyDB, TruDB, LocalSQLite
from trulens_evalchain import tru
from trulens_evalchain import tru_feedback
from IPython.display import JSON
from ipywidgets import widgets
import json

got OPENAI_API_KEY
got PINECONE_API_KEY
got PINECONE_ENV
got HUGGINGFACE_API_KEY
got SLACK_TOKEN
got SLACK_SIGNING_SECRET
got COHERE_API_KEY


openai: 0request [00:00, ?request/s]

huggingface: 0request [00:00, ?request/s]

cohere: 0request [00:00, ?request/s]

In [17]:
from langchain.callbacks import get_openai_callback
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms.base import BaseLLM
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.vectorstores import Pinecone
import pinecone

from trulens_evalchain import tru
from trulens_evalchain import tru_chain
from trulens_evalchain.keys import *
from trulens_evalchain.keys import PINECONE_API_KEY
from trulens_evalchain.keys import PINECONE_ENV

# Set up GPT-3 model
model_name = "gpt-3.5-turbo"

chain_id = "TruBot_langprompt"

# Pinecone configuration.
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_ENV  # next to api key in console
)

identity = lambda h: h

# Embedding needed for Pinecone vector db.
embedding = OpenAIEmbeddings(model='text-embedding-ada-002')  # 1536 dims
docsearch = Pinecone.from_existing_index(
    index_name="llmdemo", embedding=embedding
)
retriever = docsearch.as_retriever()

# LLM for completing prompts, and other tasks.
llm = OpenAI(temperature=0, max_tokens=128)

# Conversation memory.
memory = ConversationSummaryBufferMemory(
    max_token_limit=650,
    llm=llm,
    memory_key="chat_history",
    output_key='answer'
)

# Conversational chain puts it all together.
chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    memory=memory,
    get_chat_history=identity,
    max_tokens_limit=4096
)

# Language mismatch fix:
chain.combine_docs_chain.llm_chain.prompt.template = \
    "Use the following pieces of context to answer the question at the end " \
    "in the same language as the question. If you don't know the answer, " \
    "just say that you don't know, don't try to make up an answer.\n\n" \
    "{context}\n\n" \
    "Question: {question}\n" \
    "Helpful Answer: "

# Trulens instrumentation.
tc = tru_chain.TruChain(chain, chain_id=chain_id)

In [24]:
for query, obj in TruDB.matching_objects(tc, match=lambda q, o: len(q._path) > 0 and "llm" == q._path[-1]):
    print("LLM", query._path)

LLM ('chain', 'memory', 'llm')
LLM ('chain', 'combine_docs_chain', 'llm_chain', 'llm')
LLM ('chain', 'question_generator', 'llm')


In [ ]:
res, record = tc(dict(question="Who is Shayak?"))

In [ ]:
record

In [ ]:
hugs = tru_feedback.Huggingface()
openai = tru_feedback.OpenAI()

f_toxic = tru_feedback.Feedback(hugs.not_toxic).on_response()
f_lang_match = tru_feedback.Feedback(hugs.language_match).on(text1="prompt", text2="response")
f_relevance = tru_feedback.Feedback(openai.qs_relevance).on(question="input", statement="output")


f_lang_match = tru_feedback.Feedback(hugs.language_match).on(text1="prompt", text2="response")
f_qs_relevance = tru_feedback.Feedback(openai.qs_relevance) \
    .on(question="input", statement=Record.chain.combine_docs_chain._call.args.inputs.input_documents) \
    .on_multiple(multiarg="statement", each_query=Record.page_content)

feedbacks = tru.run_feedback_functions(chain=tc, record=record, feedback_functions=[f_qs_relevance])

In [ ]:
feedbacks

In [ ]:
for doc in TruDB.project(query=Record.chain.combine_docs_chain._call.args.inputs.input_documents, obj=record):
    print(doc)
    content = TruDB.project(query=Record.page_content, obj=doc)
    print(content)

In [ ]:
feedbacks

In [ ]:
# e = Endpoint(name="openai", rpm=120)
# print(e.pace.qsize())

In [ ]:
tru.endpoint_openai.tqdm.display()
i = 0
while True:
    # print(e.pace.qsize())
    tru.endpoint_openai.pace_me()
    # print(i)
    i+=1

In [ ]:
# tru_feedback.huggingface_language_match(prompt="Hello there?", response="How are you?")

In [ ]:
# db = LocalTinyDB("slackbot.json")
#tru.init_db("slackbot.sql")
#db = LocalSQLite("slackbot.sql.db")
db = LocalSQLite()

In [ ]:
df, dff = db.get_records_and_feedback(chain_ids=[])

In [ ]:
df

In [ ]:
from pprint import PrettyPrinter
pp = PrettyPrinter(compact=True)

for i, row in df.iterrows():
    
    display(widgets.HTML(f"<b>Question:</b> {row.input}"))
    
    display(widgets.HTML(f"<b>Answer:</b> {row.output}"))
    
    details = json.loads(eval(row.details))

    display(widgets.HTML(str(details['chain']['combine_docs_chain']['llm_chain']['prompt']['template'])))
    
    for doc in details['chain']['combine_docs_chain']['_call']['args']['inputs']['input_documents']:
        display(widgets.HTML(f"""
        <div style="border: 1px solid black; padding: 5px;">
        <b>Context chunk</b>: {doc['page_content']}
        """))

        """<br/>

        <b>source</b>: {doc['metadata']['source']}
        </div>"""

    print()


In [ ]:
df = db.select(
    Record,
    Record.record_id,
    Record.chain_id,
    Record.chain._call.args.inputs.question,
    Record.chain._call.rets.answer)

In [ ]:
df

In [ ]:
for row_id, row in df.iterrows():
    record_id = row.record_id
    chain_id = row.chain_id

    main_question = row['Record.chain._call.args.inputs.question']
    main_answer = row['Record.chain._call.rets.answer']

    print(chain_id, record_id, main_question)
    """
    
    print(question, answer)

    # Run feedback function and get value

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_not_hate_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine='moderation'
            ),
            tru_feedback.get_sentimentpositive_function(
                evaluation_choice='response',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            ),
            tru_feedback.get_relevance_function(
                evaluation_choice='both',
                provider='openai',
                model_engine='gpt-3.5-turbo'
            )
        ]
    )
    print(f"will insert overall feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)
    """
    
    # display(JSON(row.Record))
    # print(row.Record['chain'])

    model_name = "gpt-3.5-turbo"

    """
    for page in TruDB.project(query=Record.chain.combine_docs_chain._call.args.inputs.input_documents, obj=row.Record):
        answer = page['page_content']
        feedback = tru.run_feedback_function(
            main_question,
            answer,
	        [
            tru_feedback.get_qs_relevance_function(
                evaluation_choice='prompt',
                provider='openai',
                model_engine=model_name
            )]
        )
        db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    """

    feedback = tru.run_feedback_function(
        main_question, main_answer, [
            tru_feedback.get_language_match_function(
                provider='huggingface'
            )
        ]
    )
    print(f"will insert language match feedback for chain {chain_id}, record {record_id}")
    db.insert_feedback(record_id=record_id, chain_id=chain_id, feedback=feedback)

    # feedback = tru.run_feedback_function(

    #for leaf in TruDB.leafs(row.Record):
    #    print(leaf)


In [ ]:
row.record_id

In [ ]:
feedback = {'openai_hate_function': 1.849137515819166e-05,
 'openai_sentimentpositive_feedback_function': 1,
 'openai_relevance_function': 10}

In [ ]:
db.insert_feedback(2, feedback)

In [ ]:
db.select(Record, table=db.feedbacks)

In [ ]:
model_name = "gpt-3.5-turbo"
feedback = tru.run_feedback_function(
    "Who is Piotr?",
    "Piotr Mardziel works on transparency and accountability in machine learning with applications to security, privacy, and fairness. He holds Bachelor’s and Master’s degrees from the Worcester Polytechnic Institute and a PhD in computer science from University of Maryland, College Park. He has conducted post-doctoral research at Carnegie Mellon University, as well as taught classes in trustworthy machine learning at Stanford University and machine learning privacy and security at Carnegie Mellon University.",
	 [
        tru_feedback.get_qs_relevance_function(
            evaluation_choice='prompt',
            provider='openai',
            model_engine=model_name
        )])

In [ ]:
feedback